In [1]:
from web3 import Web3
from eth_account import Account
from bit import wif_to_key, PrivateKeyTestnet, PrivateKey, Key
from bit.network import NetworkAPI
import os
import json
import subprocess
from constants import *
from getpass import getpass

In [2]:
# Instantiate Web3
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8100"))

In [3]:
# Import mnemonic
mnemonic = os.getenv('MNEMONIC')

In [4]:
# Use the subprocess library to call the php file script from Python
def derive_wallets(mnemonic, coin, numderive):
    
    command = f'./derive -g --mnemonic="{mnemonic}" --cols=address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub --coin="{coin}" --numderive="{numderive}" --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    # Read data from standard output and err
    (output, err) = p.communicate()
    # This allows to wait for an output
    p_status = p.wait()
    keys = json.loads(output)
    
    return keys

In [5]:
# --coin and --numderive parameters 
coins = [ETH, BTC, BTCTEST]
numderive = 3

# Dict comprhenesion to iterate through coins
derivedWallets = {coin: derive_wallets(mnemonic, coin, numderive) for coin in coins}

In [6]:
# Fetch private keys
# derivedWallets[COIN][INDEX]['path, privkey, pubkey']

BTCprivkey = derivedWallets[BTCTEST][0]['privkey'] # pre-funded account
BTCprivkeyRecipient = derivedWallets[BTCTEST][1]['privkey'] # recipient account

ETHprivkey = derivedWallets[ETH][0]['privkey'] # pre-funded account
ETHprivkeyRecipient = derivedWallets[ETH][1]['privkey'] # recipient account 

In [7]:
# Convert the privkey string in a child key to an account object that bit or web3.py can use to transact
def privKeyToAccount(coin, privkey):
    
    if coin == BTCTEST:
        return PrivateKeyTestnet(privkey) 
    
    if coin == ETH:
        return Account.privateKeyToAccount(privkey)

In [8]:
# Create the raw, unsigned transaction that contains all metadata needed to transact
def createTx(coin, account, recipient, amount):
    
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])
    
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount})
        return {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }

In [16]:
# This will call create_tx, sign the transaction, then send it to the designated network

def sendTx(coin, account, recipient, amount):

    Tx = createTx(coin, account, recipient, amount)
    
    if coin == BTCTEST:
        
        TxSigned = account.sign_transaction(Tx)
        NetworkAPI.broadcast_tx_testnet(TxSigned)       
        
        return TxSigned
    
    elif coin == ETH:
        
        TxSigned = account.signTransaction(Tx)
        result = w3.eth.sendRawTransaction(TxSigned.rawTransaction)
        
        return result.hex()

In [10]:
# Create accounts variables
BTCaccount = privKeyToAccount(BTCTEST, BTCprivkey) # pre-funded account
ETHaccount = privKeyToAccount(ETH, ETHprivkey) # pre-funded account

In [11]:
# Create recipient accounts variables 
BTCrecipient = privKeyToAccount(BTCTEST, BTCprivkeyRecipient)

In [12]:
BTCrecipient

<PrivateKeyTestnet: mhfp7WUAS43Eb8Yd2pTnyeS4sTgdjWZ6u7>

In [13]:
# Create TX in BTC Testnet
createTx(BTCTEST, BTCaccount, 'mhfp7WUAS43Eb8Yd2pTnyeS4sTgdjWZ6u7', .0001)

'{"unspents":[{"amount":1473949,"confirmations":124,"script":"76a914a3db38f04501fffb64429a56d5c3b27eb86d66d688ac","txid":"0079f684b0870ca24042bf4a094f3f990d4fff333cf05d27fb9b2f310a57f5b3","txindex":1,"type":"p2pkh","vsize":148,"segwit":false}],"outputs":[["mhfp7WUAS43Eb8Yd2pTnyeS4sTgdjWZ6u7",10000],["mvTM4AQDawMbjG4gHRirMe2nXaf42odWQQ",1436377]]}'

In [14]:
# Send TX in BTC Testnet
sendTx(BTCTEST, BTCaccount, 'mhfp7WUAS43Eb8Yd2pTnyeS4sTgdjWZ6u7', .0001)

'0100000001b3f5570a312f9bfb275df03c33ff4f0d993f4f094abf4240a20c87b084f67900010000006b483045022100de4d8b5c5bbacb7b08fce822c448defc3eb7ddfedf06fb1961af54a98ba427fb022029b041f5658a4ae29280b60fef315620aff7e9693a21b26e63946d2ec03d25de012102d1e24b74942983fab9fc7362cc33ac42a182de514ef164ce2bc758bd95af342dffffffff0210270000000000001976a914179d2a57077f8f2874a33e56d3699570e0bc252488acd9ea1500000000001976a914a3db38f04501fffb64429a56d5c3b27eb86d66d688ac00000000'

In [13]:
from web3.middleware import geth_poa_middleware

w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [14]:
derivedWallets[ETH][1]

{'address': '0x02dB24FB0db709E3C69B8c56506D480Cb1bbB6d2',
 'index': 1,
 'path': "m/44'/60'/0'/0/1",
 'privkey': '0x668ee8dfef8abf5a79b7a775dc5270198306690bc79f0b6e1e75ed9a46b5bb08',
 'pubkey': '03db7d064f9318e7104969836da78b3a319ab6a7b281eb7fc20ff11c745e2f9975',
 'pubkeyhash': '2aa03b1744aac538a108f2d58bb3958ecae5165e',
 'xprv': 'xprvA43i49fb7fYLfHSCpNKbNFpxyZ26nfb7HVJTNQZb1NCpq9ooCiRHNbaPamSt1qjGUEFSD9AdgyPaEoAXSGAQPhv6M737tNwS1K6edMx2Qjr',
 'xpub': 'xpub6H34TfCUx36dsmWfvPrbjPmhXarbC8JxeiE4AnyCZhjohx8wkFjXvPtsS5JtTU9yoaJuFeVeHYjgxf6LKQbFqQW6CLiLX1WvfZzUf94Kh8V'}

In [17]:
sendTx(ETH, ETHaccount, '0x02dB24FB0db709E3C69B8c56506D480Cb1bbB6d2', 500000000)

'0xe781cac96ff16507cd84440fbd559f1553b8927c44d1ad1d170b7ad5ae19a040'